In [1]:
%matplotlib inline
%time from hikyuu.interactive import *

std::cout are redirected to python::stdout
std::cerr are redirected to python::stderr
2023-10-14 02:24:00.639 [HKU-I] - Using SQLITE3 BaseInfoDriver (BaseInfoDriver.cpp:58)
2023-10-14 02:24:00.640 [HKU-I] - Loading market information... (StockManager.cpp:499)
2023-10-14 02:24:00.640 [HKU-I] - Loading stock type information... (StockManager.cpp:512)
2023-10-14 02:24:00.641 [HKU-I] - Loading stock information... (StockManager.cpp:426)
2023-10-14 02:24:00.691 [HKU-I] - Loading stock weight... (StockManager.cpp:529)
2023-10-14 02:24:01.039 [HKU-I] - Loading KData... (StockManager.cpp:134)
2023-10-14 02:24:01.043 [HKU-I] - Preloading all day kdata to buffer! (StockManager.cpp:157)
2023-10-14 02:24:01.043 [HKU-I] - Preloading all week kdata to buffer! (StockManager.cpp:160)
2023-10-14 02:24:01.044 [HKU-I] - Preloading all month kdata to buffer! (StockManager.cpp:163)
2023-10-14 02:24:01.055 [HKU-I] - 0.02s Loaded Data. (StockManager.cpp:145)
Wall time: 1.09 s


# 1 利用 TM 实现简单的记账本

TradeManager对象可以理解为一个模拟的交易账户，负责交易的买/卖操作、记录交易记录以及持仓情况，也可以通过修改其买/卖操作的接口实现实盘接入。创建一个模拟交易账户，通常使用快捷创建函数 crtTM。TM对象的基本操作：

- buy  买入
- sell 卖出
- checkin 存入现金
- checkout 取出现金

可以利用 TM 实现简单的记账本，手工记录自己的操作情况，例如：

In [2]:
#创建一个初始资金10万元，起始日期2017年1月1日的模拟账户
my_tm = crtTM(init_cash=100000, date=Datetime(201701010000))

#2017年1月3日以9.11的价格买入100股
td = my_tm.buy(Datetime(201701030000), sm['sz000001'], 9.11, 100)

#查看当前资金及持仓情况
print(my_tm)

TradeManager {
  params: params[precision(int): 2, save_action(bool): 1, support_borrow_cash(bool): 0, support_borrow_stock(bool): 0, ],
  name: SYS,
  init_date: 2017-01-01 00:00:00,
  init_cash: 100000.00,
  firstDatetime: 2017-01-03 00:00:00,
  lastDatetime: 2017-01-03 00:00:00,
  TradeCostFunc: TradeCostFunc(TC_Zero, params[]),
  current cash: 99089.00,
  current market_value: 916.00,
  current short_market_value: 0.00,
  current base_cash: 100000.00,
  current base_asset: 0.00,
  current borrow_cash: 0.00,
  current borrow_asset: 0.00,
  Position: 
    SZ000001 平安银行 2017-01-03 00:00:00 1646 100.00 911.00 1100.00 189.00 20.75% 0.19%
  Short Position: 
  Borrow Stock: 
}


In [3]:
#转化为pandas的DataFrame显示当前持仓情况 
position = my_tm.get_position_list()
position.to_df()

,证券名称,买入日期,已持仓天数,持仓数量,投入金额,当前市值,盈亏金额,盈亏比例
证券代码,,,,,,,,
SZ000001,平安银行,2017-01-03,1646,100,911.0,1100.0,189.0,20.746432


In [4]:
#2017年2月21日以9.60的价格卖出100股
td = my_tm.sell(Datetime(201702210000), sm['sz000001'], 9.60)

my_tm

TradeManager {
  params: params[precision(int): 2, save_action(bool): 1, support_borrow_cash(bool): 0, support_borrow_stock(bool): 0, ],
  name: SYS,
  init_date: 2017-01-01 00:00:00,
  init_cash: 100000.00,
  firstDatetime: 2017-01-03 00:00:00,
  lastDatetime: 2017-02-21 00:00:00,
  TradeCostFunc: TradeCostFunc(TC_Zero, params[]),
  current cash: 100049.00,
  current market_value: 0.00,
  current short_market_value: 0.00,
  current base_cash: 100000.00,
  current base_asset: 0.00,
  current borrow_cash: 0.00,
  current borrow_asset: 0.00,
  Position: 
  Short Position: 
  Borrow Stock: 
}

# 2 利用 Excel 查看交易详情

使用 tocsv 方法将 TM 的交易记录、当前持仓及已平仓详细情况分别保存为 csv 文件，以便用 Excel 查看详情。

tocsv方法参数为一个指定的目录，目录必须以存在。其输出会在指定目录中，生成三个文件，“TM名称_交易记录.csv”、“TM名称_未平仓记录.csv”、“TM名称_已平仓记录.csv”。TM名称可在crtTM创建TM对象时指定，默认为“SYS”，如下图所示。

<img src="images/008_01_tocsv.png" align='left'>

In [5]:
#在 hikyuu_XXX.ini 文件中配置的临时路径中输出
my_tm.tocsv(sm.tmpdir())

使用 Excel 查看 csv，如：

<img src="images/008_02_tocsv_look.png" align="left">

# 3 使用序列化保存或重新载入已有TM对象

In [6]:
#保存至指定文件
from datetime import date
filename = "{}/my_trade/my_trade_record_{}.xml".format(sm.tmpdir(), date.today());
hku_save(my_tm, filename)

In [7]:
#载入已保存的TM对象
#filename = "{}/my_trade/my_trade_record_{}.xml".format(sm.tmpdir(), date.today())
new_my_tm = crtTM()
hku_load(new_my_tm, filename)

使用 hku_save 保存的对象，其格式为XML文件，可直接使用 XML 工具或浏览器查看：

<img src="images/008_03_pickle.png" align="left">

# 4 使用订单代理

In [8]:
#创建模拟交易账户进行回测，初始资金30万
my_tm = crtTM(init_cash=300000, date=Datetime(201701010000))

#注册实盘交易订单代理
my_tm.reg_broker(crtOB(TestOrderBroker(), False)) #TestOerderBroker是测试用订单代理对象，只打印
#my_tm.regBroker(crtOB(MailOrderBroker("smtp.sina.com", "yourmail@sina.com", "yourpwd", "receivermail@XXX.yy)))

#根据需要修改订单代理最后的时间戳，后续只有大于该时间戳时，订单代理才会实际发出订单指令
my_tm.broker_last_datetime=Datetime(201701010000)

#创建信号指示器（以5日EMA为快线，5日EMA自身的10日EMA作为慢线，快线向上穿越慢线时买入，反之卖出）
my_sg = SG_Flex(EMA(C, n=5), slow_n=10)

#固定每次买入1000股
my_mm = MM_FixedCount(1000)

#创建交易系统并运行
sys = SYS_Simple(tm = my_tm, sg = my_sg, mm = my_mm)
sys.run(sm['sz000001'], Query(-150))

买入：SZ000001  12.930  1000
卖出：SZ000001  12.480  1000
买入：SZ000001  12.940  1000
卖出：SZ000001  12.580  1000
买入：SZ000001  11.410  1000
卖出：SZ000001  11.230  1000
买入：SZ000001  11.330  1000
卖出：SZ000001  11.450  1000
买入：SZ000001  11.650  1000
卖出：SZ000001  11.690  1000
买入：SZ000001  11.400  1000
卖出：SZ000001  11.280  1000
